In [ ]:
import vector_database

In [2]:
user_prompt = "ax.legend(handles[::-1], labels[::-1], title='Line', loc='upper left')"
user_prompt = input('Enter python coding query')

In [3]:
from vector_database import qdrant,encoder

In [4]:
hits = qdrant.search(
    collection_name='conala',
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'intent': 'sort a list of strings `list`', 'snippet': 'list.sort()'} score: 0.7975705560626234
{'intent': 'sort a list of strings `list`', 'snippet': 'list.sort()'} score: 0.7975705560626234
{'intent': 'sorting the lists in list of lists `data`', 'snippet': '[sorted(item) for item in data]'} score: 0.7578674407026844


In [5]:

# define a variable to hold the search results
search_results = [(hit.payload,hit.score) for hit in hits]

In [6]:
filtered_search_results = []
score_results = []
for item in search_results:
    if item[1] not in score_results:
        filtered_search_results.append(item[0])
    score_results.append(item[1])
filtered_search_results

[{'intent': 'sort a list of strings `list`', 'snippet': 'list.sort()'},
 {'intent': 'sorting the lists in list of lists `data`',
  'snippet': '[sorted(item) for item in data]'}]

In [8]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a python coder. Your top priority is to help guide users with python code for the problem they have."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(filtered_search_results)[1:-1]}
    ]
)

##### Check how the LLM was able to create new information based on infomation retrieved from vector search.

In [ ]:
llm_output = completion.choices[0].message.content
print(llm_output)

{'intent': 'sorting the lists in list of lists `data`', 'snippet': '[sorted(item) for item in data]'}, {'intent': 'sorting the lists in list of lists `data` with custom sorting function `func`', 'snippet': '[sorted(item, key=func) for item in data]'}
